In [ ]:
import subprocess
import pandas as pd
import os
import numpy as np
from scipy.signal import savgol_filter

# Učitavanje sekvence iz fajla (FASTA format)
def ucitaj_sekvencu_iz_fajla(fajl_putanja):
    """
    Učitava sekvencu iz FASTA formata fajla, uklanja zaglavlje i vraća sekvencu kao string.
    """
    with open(fajl_putanja, 'r') as fajl:
        linije = fajl.readlines()
    
    # Preskakanje prvog reda (zaglavljem koji počinje sa ">") i spajanje ostatka u sekvencu
    sekvenca = "".join([linija.strip() for linija in linije[1:]])  # Početni red je zaglavlje
    
    return sekvenca


def parsiraj_iupred3(iupred_izlaz):
    """
    Parsira izlaz iz IUPred3 programa i klasifikuje svaku poziciju u sekvenci kao 'uređenu' ili 'neuređenu'.
    """
    print(f"IUPred3 izlaz:\n{iupred_izlaz}")  # Debug print da proverimo izlaz iz IUPred3
    
    rezultati = []
    for linija in iupred_izlaz.splitlines():
        if linija.strip() and not linija.startswith("#"):
            delovi = linija.split()
            if len(delovi) >= 3:
                pozicija = int(delovi[0])  # Pozicija
                aminokiselina = delovi[1]  # Aminokiselina
                skor = float(delovi[2])  # IUPred skor
                klasifikacija = "uređena" if skor <= 0.5 else "neuređena"
                rezultati.append((pozicija, aminokiselina, skor, klasifikacija))
    
    print(f"Parsirani rezultati: {rezultati}")  # Prikazivanje rezultata da proverimo šta dobijamo
    return rezultati


def generisi_fasta_za_prozor(prozor, indeks):
    """
    Generiše FASTA fajl za svaki prozor.
    """
    fasta_putanja = f"Prozori/prozor_{indeks}.fasta"
    with open(fasta_putanja, 'w') as f:
        f.write(f">prozor_{indeks}\n{prozor}")
    return fasta_putanja


def pokreni_iupred3_na_fajlu(fajl_putanja):
    """
    Pokreće IUPred3 na datom FASTA fajlu i vraća njegov izlaz.
    """
    rezultati = subprocess.run(
        ['/home/pc/Desktop/iupred3/iupred3.py', fajl_putanja, 'long'],  # Putanja do IUPred3 skripte
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    print(f"IUPred3 stdout:\n{rezultati.stdout}")  # Prikazujemo šta IUPred3 vraća na standardnom izlazu
    print(f"IUPred3 stderr:\n{rezultati.stderr}")  # Prikazujemo greške, ako ih ima
    return rezultati.stdout


def iupred(sequence, smoothing=None):
    """
    Pokreće IUPred3 sa sekvencom i opcionalnim parametrom za glatko filtriranje.
    """
    iupred_score = np.random.random(len(sequence))  # Ovde je mock rezultat IUPred3 (prava logika bi došla iz IUPred3 izlaza)
    
    # Ako je smoothing aktiviran, koristimo savgol_filter za glatko filtriranje
    if smoothing:
        # Proveriti da li je veličina prozora manja ili jednaka veličini sekvence
        window_length = 19  # Preporučeni prozor za savgol_filter (može biti promenjeno)
        if window_length > len(iupred_score):
            window_length = len(iupred_score)
        
        iupred_score = list(savgol_filter(iupred_score, window_length, 5))  # Parametar '5' je za polinomijalni stepen
    
    return iupred_score


def generisi_prozore_i_klasifikuj(sekvenca, min_duzina=5, max_duzina=100):
    prozori = []
    oznake_prozora = []
    duzine = []      # lista za dužinu podsekvence
    pozicije = []    # lista za pozicije u originalnoj sekvenci ("početak-kraj")
    
    # Generišemo prozore
    for duzina in range(min_duzina, max_duzina + 1):
        for i in range(len(sekvenca) - duzina + 1):
            podsekvenca = sekvenca[i:i + duzina]
            pozicija = f"{i+1}-{i+duzina}"
            duzine.append(duzina)
            pozicije.append(pozicija)
            
            # Generisanje FASTA fajla za prozor
            fasta_putanja = generisi_fasta_za_prozor(podsekvenca, i)
            
            # Pokrećemo IUPred3 na ovom FASTA fajlu
            iupred_izlaz = pokreni_iupred3_na_fajlu(fasta_putanja)
            iupred_parsirani_rezultati = parsiraj_iupred3(iupred_izlaz)
            
            # Uzimamo klasifikacije iz IUPred3 rezultata
            podsekvenca_oznake = [result[3] for result in iupred_parsirani_rezultati]
            
            # Print za debugovanje
            print(f"Prozor: {podsekvenca}")
            print(f"Klasifikacije u prozoru: {podsekvenca_oznake}")
            
            # Klasifikujemo prozor kao većinski 'uređena' ili 'neuređena'
            if podsekvenca_oznake.count("uređena") > len(podsekvenca_oznake) / 2:
                oznake_prozora.append("uređena")
            else:
                oznake_prozora.append("neuređena")
            
            prozori.append(podsekvenca)
    
    return duzine, pozicije, prozori, oznake_prozora


# Učitavanje sekvence iz fajla
fajl_putanja = "YP_138523.1.fasta"  # Putanja do FASTA fajla
sekvenca = ucitaj_sekvencu_iz_fajla(fajl_putanja)

# Generisanje prozora i klasifikacija na osnovu IUPred3
duzine, pozicije, prozori, oznake_prozora = generisi_prozore_i_klasifikuj(sekvenca)

# Kreiranje DataFrame sa rezultatima
df_sekvence = pd.DataFrame({
    "Duzina": duzine,
    "Pozicija": pozicije,
    "Sekvenca": prozori,
    "IUPRED3 oznaka": oznake_prozora
})

# Prikazivanje rezultata
print(df_sekvence.head())

# Čuvanje rezultata u CSV fajl
df_sekvence.to_csv("Klasifikovane_sekvence_IUPRED3.csv", index=False)
